In [85]:
import sqlite3 as sqlite
import pandas as pd
import typing as List
import json

#Acessando o banco de dados
conn = sqlite.connect('clicksign_test.db')
cursor = conn.cursor()

#Tratando os dados da tabela country
#Renomeando a tabela country para aplicar o tratamente e recria-la
cursor.execute("""ALTER TABLE country RENAME TO country_legacy;""")

#Executando o tratamento
cursor.execute(""" CREATE TABLE country AS
    WITH 
    
    -- Retirando dados nulos. Esta tabela é normalizada não faz sentido ter dados nulos para id ou name.
    excludes_nulls as (
        SELECT 
            * 
        FROM country_legacy
            WHERE name is not null),
            
    -- Há ids duplicados para dados com o mesmo significados em línguas diferentes.
    -- Eliminando os ids duplicado em português.
    excludes_portuguese_name as (
        SELECT 
            *
        FROM excludes_nulls
        WHERE name not in ('Bélgica','Inglaterra','França','Alemanha','Itália'))
    
    -- Transformando o id em inteiro e colocando o nome em uppercase.
    SELECT 
        cast(id as integer) as id,
        upper(name) as name
    FROM excludes_portuguese_name
    
""")

cursor.execute(""" DROP TABLE IF EXISTS country_legacy """)

#Tratando os dados da tabela league
#Renomeando a tabela league para aplicar o tratamente e recria-la
cursor.execute("""ALTER TABLE league RENAME TO league_legacy;""")

cursor.execute("""
        -- Colocando a lista em ordem de id e colocando o nome da liga em uppercase
        CREATE TABLE league as
        SELECT 
            id,
            country_id,
            upper(name) as name
        FROM league_legacy
        ORDER BY id;
""")
cursor.execute(""" DROP TABLE IF EXISTS league_legacy """)

#Como há um registro na tabela league do Brazil e Angola que não há em country
#Será feito um apdate na tabela country do id e do nome destes dois países
insert_query = 'INSERT INTO country VALUES(?,?);'
values = [[44560,'BRAZIL'],[51220,'ANGOLA']]
cursor.executemany(insert_query, values)

#Tratando os dados da tabela player
#Como o campo birthday está como datetime porém a informação relevante é a data, transformar a coluna em apenas date
#Deixar o nome dos jogadores em uppercase
cursor.execute("""ALTER TABLE player RENAME TO player_legacy;""")

cursor.execute(""" 
    CREATE TABLE player AS
    SELECT 
        id, 
        player_api_id, 
        upper(player_name) as player_name, 
        player_fifa_api_id, 
        replace(birthday, ' 00:00:00', '') as birthday, 
        height, 
        weight 
    FROM player_legacy """)

cursor.execute(""" DROP TABLE IF EXISTS player_legacy """)


#Tratando os dados da tabela team
# Deixando o nome do time em uppercase
cursor.execute("""ALTER TABLE team RENAME TO team_legacy;""")

cursor.execute(""" 
    CREATE TABLE team AS
    SELECT 
        id, 
        team_api_id, 
        team_fifa_api_id, 
        upper(team_long_name) as team_long_name, 
        team_short_name 
    FROM team_legacy """)

cursor.execute(""" DROP TABLE IF EXISTS team_legacy """)


#Tratamento da tabela  team_attributes
#Transfprmando Json em colunas
#importando arquivo csv
team_attributes : pd.DataFrame = pd.read_csv('../csv_files/Team_Attributes.csv',index_col = 0) 
    
#Transformando a coluna json em colunas
row_list : List = []
for i in range(0, team_attributes.shape[0]):
    row_list.append(json.loads(team_attributes.Team_Attributes[i]))
team_attributes = pd.DataFrame(row_list)

cursor.execute("""DROP TABLE IF EXISTS team_attributes;""")
cursor.execute("""
    CREATE TABLE team_attributes_legacy(
        id, team_fifa_api_id, team_api_id, date, buildUpPlaySpeed, buildUpPlaySpeedClass, buildUpPlayDribbling,
        buildUpPlayDribblingClass, buildUpPlayPassing, buildUpPlayPassingClass, buildUpPlayPositioningClass,
        chanceCreationPassing, chanceCreationPassingClass, chanceCreationCrossing, chanceCreationCrossingClass,
        chanceCreationShooting, chanceCreationShootingClass, chanceCreationPositioningClass, defencePressure,
        defencePressureClass, defenceAggression, defenceAggressionClass, defenceTeamWidth, defenceTeamWidthClass, 
        defenceDefenderLineClass
        ); """)

insert_query = '''INSERT INTO team_attributes_legacy VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);'''
cursor.executemany(insert_query, team_attributes.values.tolist())

#Tratando a tipagem das colunas e deixando os campos categorias em uppercase
cursor.execute("""
            CREATE TABLE team_attributes AS 
            SELECT  
                cast(id as integer)                AS id, 
                cast(team_fifa_api_id 
                    as integer)                    AS team_fifa_api_id, 
                cast(team_api_id as integer)       AS team_api_id, 
                replace(date, ' 00:00:00', '')     AS date, 
                cast(buildUpPlaySpeed 
                    as integer)                    AS build_up_play_speed, 
                upper(buildUpPlaySpeedClass)       AS build_up_play_speed_class, 
                cast(buildUpPlayDribbling 
                    as integer)                    AS build_up_play_dribbling,
                upper(buildUpPlayDribblingClass)   AS build_up_play_dribbling_class, 
                cast(buildUpPlayPassing 
                    as integer)                    AS build_up_play_passing, 
                upper(buildUpPlayPassingClass)     AS build_up_play_passing_class, 
                upper(buildUpPlayPositioningClass) AS build_up_play_positioning_class,
                cast(chanceCreationPassing 
                    as integer)                    AS chance_creation_passing, 
                upper(chanceCreationPassingClass)  AS chance_creation_passing_class, 
                cast(chanceCreationCrossing 
                    as integer)                    AS chance_creation_crossing, 
                upper(chanceCreationCrossingClass) AS chance_creation_crossing_class,
                cast(chanceCreationShooting 
                    as integer)                    AS chance_creation_shooting, 
                upper(chanceCreationShootingClass) AS chance_creation_shooting_class, 
                upper(chanceCreationPositioningClass) AS chance_creation_positioning_class, 
                cast(defencePressure as integer)   AS defence_pressure,
                upper(defencePressureClass)        AS defence_pressure_class, 
                cast(defenceAggression as integer) AS defence_aggression, 
                upper(defenceAggressionClass)      AS defence_aggression_class, 
                cast(defenceTeamWidth as integer)  AS defence_team_width, 
                upper(defenceTeamWidthClass)       AS defence_team_width_class,
                upper(defenceDefenderLineClass)    AS defence_defender_line_class
            FROM team_attributes_legacy
""")

cursor.execute(""" DROP TABLE IF EXISTS team_attributes_legacy """)


#importando arquivo csv
player_attributes : pd.DataFrame = pd.read_csv('../csv_files/Player_Attributes.csv',index_col = 0) 
    
#Transformando a coluna json em colunas
row_list : List = []
for i in range(0, player_attributes.shape[0]):
    row_list.append(json.loads(player_attributes.Player_Attributes[i]))
player_attributes = pd.DataFrame(row_list)

#Criando a tabela player_attributes_columns
cursor.execute("""DROP TABLE IF EXISTS player_attributes;""")
cursor.execute("""
    CREATE TABLE player_attributes_legacy(
        id, player_fifa_api_id, player_api_id, date, overall_rating, potential, preferred_foot,
        attacking_work_rate, defensive_work_rate, crossing, finishing, heading_accuracy, short_passing,
        volleys, dribbling, curve, free_kick_accuracy, long_passing, ball_control, acceleration, sprint_speed,
        agility, reactions, balance, shot_power, jumping, stamina, strength, long_shots, aggression,
        interceptions, positioning, vision, penalties, marking, standing_tackle, sliding_tackle, 
        gk_diving, gk_handling, gk_kicking, gk_positioning, gk_reflexes); """)

insert_query = '''INSERT INTO player_attributes_legacy VALUES(
                    ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,
                    ?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?
                 );'''
cursor.executemany(insert_query, player_attributes.values.tolist())

# Como os valores númericos são todos inteiros troquei os campos float por int
# Coloquei os valores categóricos em uppercase
cursor.execute(""" 
    CREATE TABLE player_attributes AS
    SELECT 
        cast(id as integer)             AS id, 
        cast(player_fifa_api_id 
             as integer)                AS player_fifa_api_id,
        cast(player_api_id as integer)  AS player_api_id, 
        replace(date, ' 00:00:00', '')  AS date,
        cast(overall_rating as integer) AS overall_rating,
        cast(potential as integer)      AS potential,
        upper(preferred_foot)           AS preferred_foot ,
        upper(attacking_work_rate)      AS attacking_work_rate,
        upper(defensive_work_rate)      AS defensive_work_rate,
        cast(crossing as integer)       AS crossing,
        cast(finishing as integer)      AS finishing,
        cast(heading_accuracy 
             as integer)                AS heading_accuracy,
        cast(short_passing as integer)  AS short_passing,
        cast(volleys as integer)        AS volleys, 
        cast(dribbling as integer)      AS dribbling,
        cast(curve as integer)          AS curve,
        cast(free_kick_accuracy 
            as integer)                 AS free_kick_accuracy,
        cast(long_passing as integer)   AS long_passing,
        cast(ball_control as integer)   AS ball_control, 
        cast(acceleration as integer)   AS acceleration,
        cast(sprint_speed as integer)   AS sprint_speed,
        cast(agility as integer)        AS agility, 
        cast(reactions as integer)      AS reactions,
        cast(balance as integer)        AS balance,
        cast(shot_power as integer)     AS shot_power,
        cast(jumping as integer)        AS jumping,
        cast(stamina as integer)        AS stamina,
        cast(strength as integer)       AS strength,
        cast(long_shots as integer)     AS long_shots, 
        cast(aggression as integer)     AS aggression,
        cast(interceptions as integer)  AS interceptions,
        cast(positioning as integer)    AS positioning,
        cast(vision as integer)         AS vision,
        cast(penalties as integer)      AS penalties,
        cast(marking as integer)        AS marking,
        cast(standing_tackle 
            as integer)                 AS standing_tackle,
        cast(sliding_tackle as integer) AS sliding_tackle, 
        cast(gk_diving as integer)      AS gk_diving, 
        cast(gk_handling as integer)    AS gk_handling,
        cast(gk_kicking as integer)     AS gk_kicking,
        cast(gk_positioning as integer) AS gk_positioning, 
        cast(gk_reflexes as integer)    AS gk_reflexes
    FROM player_attributes_legacy

""")

cursor.execute("""DROP TABLE IF EXISTS player_attributes_legacy;""")

# Corrigindo as tabelas player_attributes_modified e team_attributes_modified com os tipos corrigidos
cursor.execute("DROP TABLE IF EXISTS player_attributes_modified;")
cursor.execute("""
    CREATE TABLE player_attributes_modified AS
    SELECT 
        player.id AS player_id,
        player.player_api_id, 
        player_name, 
        player.player_fifa_api_id, 
        birthday, 
        height, 
        weight, 
        player_attributes.id AS player_attributes_id, 
        date, 
        overall_rating,
        potential,
        preferred_foot,
        attacking_work_rate,
        defensive_work_rate,
        crossing,
        finishing,
        heading_accuracy,
        short_passing,
        volleys, 
        dribbling,
        curve,
        free_kick_accuracy,
        long_passing,
        ball_control,
        acceleration,
        sprint_speed,
        agility, 
        reactions,
        balance,
        shot_power,
        jumping,
        stamina,
        strength,
        long_shots,
        aggression,
        interceptions, 
        positioning,
        vision,
        penalties,
        marking,
        standing_tackle,
        sliding_tackle,
        gk_diving, 
        gk_handling,
        gk_kicking, 
        gk_positioning, 
        gk_reflexes
    FROM player
    INNER JOIN player_attributes
        ON player_attributes.player_api_id = player.player_api_id;""")


cursor.execute("DROP TABLE IF EXISTS team_attributes_modified;")
cursor.execute("""
    CREATE TABLE team_attributes_modified as
    SELECT 
        team.id as team_id, 
        team.team_api_id, 
        team.team_fifa_api_id, 
        team_long_name, 
        team_short_name, 
        team_attributes.id as team_attribute_id, 
        date, 
        build_up_play_speed, 
        build_up_play_speed_class, 
        build_up_play_dribbling,
        build_up_play_dribbling_class, 
        build_up_play_passing, 
        build_up_play_passing_class, 
        build_up_play_positioning_class,
        chance_creation_passing, 
        chance_creation_passing_class, 
        chance_creation_crossing, 
        chance_creation_crossing_class,
        chance_creation_shooting, 
        chance_creation_shooting_class, 
        chance_creation_positioning_class, 
        defence_pressure,
        defence_pressure_class, 
        defence_aggression, 
        defence_aggression_class, 
        defence_team_width, 
        defence_team_width_class,
        defence_defender_line_class
    FROM team
    INNER JOIN team_attributes
        ON team_attributes.team_api_id = team.team_api_id;
""")



#retirar da tabela match as libas em que os players id são nulos.
cursor.execute("""ALTER TABLE match RENAME TO match_legacy;""")
cursor.execute("""
    CREATE TABLE match as
    SELECT 
        id, 
        country_id, 
        league_id, 
        season, 
        stage, 
        replace(date, ' 00:00:00','') AS date, 
        match_api_id, 
        home_team_api_id, 
        away_team_api_id, 
        home_team_goal, 
        away_team_goal, 
        cast(home_player_1 as integer) AS home_player_1, 
        cast(home_player_2 as integer) AS home_player_2, 
        cast(home_player_3 as integer) AS home_player_3, 
        cast(home_player_4 as integer) AS home_player_4, 
        cast(home_player_5 as integer) AS home_player_5, 
        cast(home_player_6 as integer) AS home_player_6, 
        cast(home_player_7 as integer) AS home_player_7, 
        cast(home_player_8 as integer) AS home_player_8, 
        cast(home_player_9 as integer) AS home_player_9, 
        cast(home_player_10 as integer) AS home_player_10, 
        cast(home_player_11 as integer) AS home_player_11, 
        cast(away_player_1 as integer) AS away_player_1, 
        cast(away_player_2 as integer) AS away_player_2, 
        cast(away_player_3 as integer) AS away_player_3, 
        cast(away_player_4 as integer) AS away_player_4, 
        cast(away_player_5 as integer) AS away_player_5, 
        cast(away_player_6 as integer) AS away_player_6, 
        cast(away_player_7 as integer) AS away_player_7, 
        cast(away_player_8 as integer) AS away_player_8, 
        cast(away_player_9 as integer) AS away_player_9, 
        cast(away_player_10 as integer) AS away_player_10, 
        cast(away_player_11 as integer) AS away_player_11,
        goal, 
        shoton, 
        shotoff    AS shot_off,
        foulcommit AS foul_commit, 
        card, 
        cross, 
        corner, 
        possession, 
        B365H      AS b365H, 
        B365D      AS b365d, 
        B365A      AS b365A, 
        BWH        AS bwh, 
        BWD        AS bwd, 
        BWA        AS bwa,
        IWH        AS iwh, 
        IWD        AS iwd, 
        IWA        AS iwa, 
        LBH        AS lbh, 
        LBD        AS lbd, 
        LBA        AS lba, 
        PSH        AS psh, 
        PSD        AS psd, 
        PSA        AS psa, 
        WHH        AS whh, 
        WHD        AS whd, 
        WHA        AS wha, 
        SJH        AS sjh, 
        SJD        AS sjd, 
        SJA        AS sja, 
        VCH        AS vch, 
        VCD        AS vcd, 
        VCA        AS vca, 
        GBH        AS gbh, 
        GBD        AS gbd, 
        GBA        AS gba, 
        BSH        AS bsh, 
        BSD        AS bsd, 
        BSA        AS bsa 
    FROM match_legacy
    WHERE TRUE
        AND (country_id IS NOT NULL OR league_id IS NOT NULL)
        AND home_team_api_id IS NOT NULL
        AND away_team_api_id IS NOT NULL
        AND home_player_1 IS NOT NULL
        AND home_player_2 IS NOT NULL 
        AND home_player_3 IS NOT NULL 
        AND home_player_4 IS NOT NULL 
        AND home_player_5 IS NOT NULL 
        AND home_player_6 IS NOT NULL 
        AND home_player_7 IS NOT NULL 
        AND home_player_8 IS NOT NULL 
        AND home_player_9 IS NOT NULL 
        AND home_player_10 IS NOT NULL 
        AND home_player_11 IS NOT NULL 
        AND away_player_1 IS NOT NULL
        AND away_player_2 IS NOT NULL 
        AND away_player_3 IS NOT NULL 
        AND away_player_4 IS NOT NULL 
        AND away_player_5 IS NOT NULL 
        AND away_player_6 IS NOT NULL 
        AND away_player_7 IS NOT NULL 
        AND away_player_8 IS NOT NULL 
        AND away_player_9 IS NOT NULL 
        AND away_player_10 IS NOT NULL 
        AND away_player_11 IS NOT NULL
  
""")
cursor.execute("""DROP TABLE IF EXISTS match_legacy;""")


#Finalizando a conexão
conn.commit()
conn.close()